In [12]:
from ss2hcsp.sl.sl_diagram import SL_Diagram
from ss2hcsp.sl.get_hcsp import get_hcsp

In [5]:
location = "./Examples/UnitDelay.xml"
diagram = SL_Diagram(location=location)
diagram.parse_xml()
diagram.add_line_name()
diagram.comp_inher_st()
print(diagram)

Blocks:
Constant: Constant[value = 1, out = [[x0]]]
Sum: Add[in = [x0, x1], out = [[x2]], st = -1]
Unit Delay: UnitDelay[init = 0, in = [x2], out = [[x1]], delay = -1]


In [9]:
discrete, continuous, sf_charts, buffers = diagram.seperate_diagram()

In [10]:
discrete

[[<ss2hcsp.sl.Continuous.constant.Constant at 0x230f2ae93c8>,

In [15]:
continuous

[[<ss2hcsp.sl.Discrete.unit_delay.UnitDelay at 0x230f2da0668>]]

In [16]:
sf_charts

[]

In [17]:
buffers

[]

In [18]:
get_hcsp(discrete, continuous, sf_charts, buffers)

AssertionError: 